In [1]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_chroma import Chroma

DATA_PATH = "data"

# Document LOADER
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

# Split Documents into Chunks
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser



generate_queries = (
    prompt_perspectives 
    | OllamaLLM(model="llama3.2")
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [3]:
from langchain_ollama import OllamaEmbeddings

#### INDEXING ####
documents = load_documents()
splits = split_documents(documents)

from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text"))

# Define Retriever 
retriever = vectorstore.as_retriever()

In [4]:
from langchain.load import dumps, loads

# Define function to remove duplicates from the multiple queries outputs
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is perimeter fit?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)


C:\Users\eriko\AppData\Local\Temp\ipykernel_15512\795381699.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


16

In [5]:
from operator import itemgetter
from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="llama3.2")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Perimeter Fit is a module mentioned in the text, which appears to be a part of an object segmentation algorithm. Its purpose is to refine rough class maps (CAMs) by removing lower-confidence pixels while keeping positively classified pixels. This process uses a perimeter map generated from image segmentation techniques such as Quickshift or SLIC. The goal is to shrink the original CAM size without extending it, which can be incorporated into any existing object segmentation algorithm.'